In [7]:
import timeit
import pandas as pd
from mupa.multiprocessing import multiprocessor

pmp = multiprocessor()

## Grab Data for testing 
df_1k = pd.read_csv('movie_dialog_pairs.csv')[:1000]
df_10k = pd.concat([df_1k] * 10)
df_100k = pd.concat([df_1k] * 100)
df_1m =  pd.concat([df_1k] * 1000)
df_10m =  pd.concat([df_1k] * 10000)

In [8]:
## some functions for trial
def has_long_words(_string):
    _list = [len(word) for word in _string.split()]

    if len(_list) > 0:
        return True if max(_list) > 5 else False
    else:
        return False

def has_long_words_applicator(df):
        ## ensures new_col exists
        ## create new column with function passed
        df['parent_contains_longword'] = df['parent'].apply(has_long_words)
        ## return the frame
        return df

def contains_test_word(_string):
    test_words = ['hello', 'world', 'never', 'going', 'to', 'give', 'you', 'up']
    _string = _string.lower().split()

    for word in _string:
        if word in test_words:
            return True
    return False

def test_word_applicator(df):
    ## ensures new_col exists
    ## create new column with function passed
    df['parent_contains_test_word'] = df['parent'].apply(contains_test_word)
    ## return the frame
    return df


In [11]:
# demonstration of application
_ = pmp.parallel_apply(df_1k, has_long_words_applicator)
_['parent_contains_longword'].value_counts()

True     243768
False     60328
Name: parent_contains_longword, dtype: int64

In [10]:
_ = pmp.parallel_apply(df_1k, test_word_applicator)
_['parent_contains_test_word'].value_counts()

False    180622
True     123474
Name: parent_contains_test_word, dtype: int64

In [16]:
def lw_df_1k():
    pmp.parallel_apply(df_1k, has_long_words_applicator)

def tw_df_1k():
    pmp.parallel_apply(df_1k, test_word_applicator)
def tw_vanilla_df_1k():
    df_1k['parent_contains_test_word'] = df_1k['parent'].apply(contains_test_word)
    
def lw_vanilla_df_1k():
    df_1k['parent_contains_test_word'] = df_1k['parent'].apply(has_long_words)

print('Vanilla Functions outperform on small dataframes')
print(timeit.timeit(lw_vanilla_df_1k, number = 2), 
      timeit.timeit(tw_vanilla_df_1k, number = 2))

print(timeit.timeit(lw_df_1k, number = 2), 
      timeit.timeit(tw_df_1k, number = 2))

2.2197076520023984 1.7246787539988873
13.29890669499946 12.736757097001828


In [17]:

def lw_df_10m():
    return pmp.parallel_apply(df_10m, has_long_words_applicator)

def tw_df_10m():
    return pmp.parallel_apply(df_10m, test_word_applicator)

def tw_vanilla_df_10m():
    df_10m['parent_contains_test_word'] = df_10m['parent'].apply(contains_test_word)
    
def lw_vanilla_df_10m():
    df_10m['parent_contains_test_word'] = df_10m['parent'].apply(has_long_words)

print('Vanilla Functions perform way better on large dataframes')
print(timeit.timeit(lw_vanilla_df_10m, number = 2), 
      timeit.timeit(tw_vanilla_df_10m, number = 2))

print(timeit.timeit(lw_df_10m, number = 2), 
      timeit.timeit(tw_df_10m, number = 2))

63.32220673899792 47.51651924100224
221.6061473910013 203.9420653490015
